In [16]:
import torch
from models.vgg_cifar import MaskVGG,VGG
import torch
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import os
from lib.utils import AverageMeter,accuracy
import torch.nn as nn
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
# from model.googlenet import Inception
# from utils.options import args
# import utils.common as utils

import os
import time
import copy
import sys
import random
import numpy as np
import heapq
# from data import cifar10, cifar100, imagenet
from importlib import import_module

device = torch.device(f"cuda:0") if torch.cuda.is_available() else 'cpu'

In [162]:

vgg = VGG('vgg16').cuda()
vgg.load_state_dict(torch.load('C:\\Users\\lenovo\\Desktop\\cacp\\amc_vgg\\checkpoints\\vgg16_cifar10.pt')['state_dict'])

<All keys matched successfully>

In [169]:
list(vgg.modules())[2]

Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [22]:
n,c,H,W = list(vgg.modules())[2].weight.size()
# c
# H

In [3]:
def get_split_dataset(dset_name, batch_size, n_worker, val_size, data_root='../data',
                      use_real_val=False, shuffle=True):
    '''
        split the train set into train / val for rl search
    '''
    if shuffle:
        index_sampler = SubsetRandomSampler
    else:  # every time we use the same order for the split subset
        class SubsetSequentialSampler(SubsetRandomSampler):
            def __iter__(self):
                return (self.indices[i] for i in torch.arange(len(self.indices)).int())
        index_sampler = SubsetSequentialSampler

    print('=> Preparing data: {}...'.format(dset_name))
    if dset_name == 'cifar10':
        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
        trainset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=True, transform=transform_train)
        if use_real_val:  # split the actual val set
            valset = torchvision.datasets.CIFAR10(root=data_root, train=False, download=True, transform=transform_test)
            n_val = len(valset)
            assert val_size < n_val
            indices = list(range(n_val))
            np.random.shuffle(indices)
            _, val_idx = indices[val_size:], indices[:val_size]
            train_idx = list(range(len(trainset)))  # all train set for train
        else:  # split the train set
            valset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=True, transform=transform_test)
            n_train = len(trainset)
            indices = list(range(n_train))
            # now shuffle the indices
            np.random.shuffle(indices)
            assert val_size < n_train
            train_idx, val_idx = indices[val_size:], indices[:val_size]

        train_sampler = index_sampler(train_idx)
        val_sampler = index_sampler(val_idx)

        train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, sampler=train_sampler,
                                                   num_workers=n_worker, pin_memory=True)
        val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, sampler=val_sampler,
                                                 num_workers=n_worker, pin_memory=True)
        n_class = 10
        
    elif dset_name == 'imagenet':
        train_dir = os.path.join(data_root, 'train')
        val_dir = os.path.join(data_root, 'val')
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                         std=[0.229, 0.224, 0.225])
        input_size = 224
        train_transform = transforms.Compose([
                transforms.RandomResizedCrop(input_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                normalize,
            ])
        test_transform = transforms.Compose([
                transforms.Resize(int(input_size/0.875)),
                transforms.CenterCrop(input_size),
                transforms.ToTensor(),
                normalize,
            ])

        trainset = datasets.ImageFolder(train_dir, train_transform)
        if use_real_val:
            valset = datasets.ImageFolder(val_dir, test_transform)
            n_val = len(valset)
            assert val_size < n_val
            indices = list(range(n_val))
            np.random.shuffle(indices)
            _, val_idx = indices[val_size:], indices[:val_size]
            train_idx = list(range(len(trainset)))  # all trainset
        else:
            valset = datasets.ImageFolder(train_dir, test_transform)
            n_train = len(trainset)
            indices = list(range(n_train))
            np.random.shuffle(indices)
            assert val_size < n_train
            train_idx, val_idx = indices[val_size:], indices[:val_size]

        train_sampler = index_sampler(train_idx)
        val_sampler = index_sampler(val_idx)

        train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler,
                                                   num_workers=n_worker, pin_memory=True)
        val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, sampler=val_sampler,
                                                 num_workers=n_worker, pin_memory=True)

        n_class = 1000
    else:
        raise NotImplementedError

    return train_loader, val_loader, n_class

In [4]:
def _validate(val_loader, model, verbose=True):
        '''
        Validate the performance on validation set
        :param val_loader:
        :param model:
        :param verbose:
        :return:
        '''
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
        top5 = AverageMeter()

        criterion = nn.CrossEntropyLoss().cuda()
        # switch to evaluate mode
        model.eval()
        end = time.time()

        t1 = time.time()
        with torch.no_grad():
            for i, (input, target) in enumerate(val_loader):
                target = target.cuda(non_blocking=True)
                input_var = torch.autograd.Variable(input).cuda()
                target_var = torch.autograd.Variable(target).cuda()

                # compute output
                output = model(input_var)
                loss = criterion(output, target_var)

                # measure accuracy and record loss
                prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
                losses.update(loss.item(), input.size(0))
                top1.update(prec1.item(), input.size(0))
                top5.update(prec5.item(), input.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()
        t2 = time.time()
        if verbose:
            print('* Test loss: %.3f    top1: %.3f    top5: %.3f    time: %.3f' %
                  (losses.avg, top1.avg, top5.avg, t2 - t1))
        # if self.acc_metric == 'acc1':
        #     return top1.avg
        # elif self.acc_metric == 'acc5':
        #     return top5.avg
        # else:
        #     raise NotImplementedError

In [5]:

train_loader, val_loader, n_class = get_split_dataset('cifar10', 50,
                                                                        0, 5000,
                                                                        data_root='C:\\Users\\lenovo\\dataset\\cifar',
                                                                        use_real_val=True,
                                                                        shuffle=False)  # same sampling
_validate(val_loader,vgg)

=> Preparing data: cifar10...
Files already downloaded and verified
Files already downloaded and verified
* Test loss: 0.317    top1: 92.920    top5: 99.740    time: 4.354


In [7]:
# def vgg_masked(vgg_origin,strategy):
#     from models.vgg_cifar import MaskVGG
#     masked_vgg = MaskVGG('vgg16',strategy)
    
#     orimo_ls = list(vgg_origin.modules())
    
#     for i,m in enumerate(masked_vgg.modules()):
#         if type(m) in [nn.Conv2d,nn.BatchNorm2d,nn.Linear,nn.AvgPool2d,nn.MaxPool2d,nn.ReLU]:
#             # type
#             ty = type(m)

#             # conv
#             if ty == nn.Conv2d:
#                 m.weight.data.copy_(orimo_ls[i].weight.data)
#                 m.bias.data.copy_(orimo_ls[i].bias.data)
#             # bn2d
#             elif ty == nn.BatchNorm2d:
#                 m.weight.data.copy_(orimo_ls[i].weight.data)
#                 m.bias.data.copy_(orimo_ls[i].bias.data)
#                 m.running_mean.data.copy_(orimo_ls[i].running_mean.data)
#                 m.running_var.data.copy_(orimo_ls[i].running_var.data)
#             elif ty == nn.Linear:
#                 # linear
#                 m.weight.data.copy_(orimo_ls[i].weight.data)
#             else:# maxpool,avgpool,relu don't need params
#                 pass
#     masked_vgg = masked_vgg.cuda()
#     return masked_vgg

In [8]:
# mask = vgg_masked(vgg,strategy=[1.0]*13)
# _validate(val_loader,mask)

In [9]:
all_idx = []
all_ops= []
prunable_idx = []
prunable_ops = []
orgin_channel = []

In [10]:

for i,m in enumerate(vgg.modules()):
    if type(m) in [nn.Conv2d,nn.BatchNorm2d,nn.Linear,nn.AvgPool2d,nn.MaxPool2d,nn.ReLU]:
        all_idx.append(i)
        all_ops.append(m)
        if type(m) in [nn.Conv2d]:
            prunable_idx.append(i)
            prunable_ops.append(m)
            orgin_channel.append(m.out_channels)

In [11]:
def idx2idxx(idx):
    return prunable_idx[idx]
def idxx2idx(idxx):
    return prunable_idx.index(idxx)


In [12]:
def preprocess_get_mask(select,method = 'l1'):
    mask = []
    for i,a in enumerate(select):
        c = orgin_channel[i]
        d = int(c * a)
        mask_ = np.zeros(c,bool)
        weight = prunable_ops[i].weight.data.cpu().numpy()
        if method == 'l1':
            importance = np.abs(weight).sum((1, 2, 3))
            sorted_idx = np.argsort(-importance)  # sum magnitude along C_in, sort descend
            preserve_idx = sorted_idx[:d]  # to preserve index
            mask_[preserve_idx] = True
        mask.append(mask_)
    return mask

In [13]:
prunable_idx = []
prunable_ops = []
layer_type_dict = {}
org_channels = {}
conv_buffer_dict = {} # layer after the conv
all_idx = []
buffer_conv_map = {}

i=0
buffer_temp_idx = []
modules = list(vgg.modules())
n = len(modules)
while i < n :
    m = modules[i]

    if type(m) not in [nn.Conv2d,nn.BatchNorm2d,nn.Linear,nn.AvgPool2d,nn.MaxPool2d,nn.ReLU]:
        i+=1
        continue
    else:
        assert type(m) == torch.nn.modules.conv.Conv2d
        prunable_ops.append(m)
        prunable_idx.append(i)
        layer_type_dict[i] = type(m)
        org_channels[i] = (m.in_channels) 
        
        buffer_temp_idx = []
        while i != n-1:
            i+=1
            bu = modules[i]
            if type(bu) is torch.nn.modules.conv.Conv2d:
                i-=1
                break
            buffer_temp_idx.append(i)
        conv_buffer_dict[prunable_idx[-1]] = copy.deepcopy(buffer_temp_idx)
        for j in buffer_temp_idx:
            buffer_conv_map[j] = prunable_idx[-1]
    i+=1


In [14]:
def pruned_model(origin_model,pruned_model,all_mask):
    m_list = list(origin_model.modules())
    mp_list = list(pruned_model.modules())
    st = time.time()
    for idx,idxx in enumerate(prunable_idx):
        
        # replace conv first
        mask = all_mask[idx]
        weight = m_list[idxx].weight.data.cpu().numpy()
        bias = m_list[idxx].bias.data.cpu().numpy()
        
        mask_weight = None
        if idx == 0:
            mask_weight = weight[mask,:,:,:]
        else:
            input_mask = all_mask[idx-1]
            # select input
            mask_weight = weight[:,input_mask,:,:].reshape(weight.shape[0],-1,weight.shape[2],weight.shape[3])
            # select output
            mask_weight = mask_weight[mask,:,:,:].reshape(-1,mask_weight.shape[1],mask_weight.shape[2],mask_weight.shape[3])
        mask_bias = bias[mask]
        mp = mp_list[idxx]
        mp.weight.data.copy_(torch.from_numpy(mask_weight).cuda())
        mp.bias.data.copy_(torch.from_numpy(mask_bias).cuda())

        # replace other layers
        buffer = conv_buffer_dict[idxx]
        for buffer_idx in buffer:
            m = m_list[buffer_idx]
            mp = mp_list[buffer_idx]
            if type(m) == nn.BatchNorm2d:
                mp.weight.data.copy_(torch.from_numpy(m.weight.data.cpu().numpy()[mask]).cuda())
                mp.bias.data.copy_(torch.from_numpy(m.bias.data.cpu().numpy()[mask]).cuda())
                mp.running_mean.data.copy_(torch.from_numpy(m.running_mean.data.cpu().numpy()[mask]).cuda())
                mp.running_var.data.copy_(torch.from_numpy(m.running_var.data.cpu().numpy()[mask]).cuda())
            elif type(m) == nn.Linear:
                mp.weight.data.copy_(torch.from_numpy(m.weight.data.cpu().numpy()[:,mask]).cuda())
    ed = time.time()
#     print(f'replace cost {ed-st}s')

In [15]:
def get_masked_vgg(select):
    mask = preprocess_get_mask(select)
    from models.vgg_cifar import MaskVGG
    masked_vgg = MaskVGG('vgg16',select).cuda()
    pruned_model(vgg,masked_vgg,mask)
    return masked_vgg

In [19]:
def get_bf_af_acc(select):
    mask = preprocess_get_mask(select)
    from models.vgg_cifar import MaskVGG
    masked_vgg = MaskVGG('vgg16',select).cuda()
    _validate(val_loader,masked_vgg)
    pruned_model(vgg,masked_vgg,mask)
    _validate(val_loader,masked_vgg)

# select = [0.75, 0.375, 0.8125, 1.0, 0.8125, 0.53125, 0.71875, 0.875, 0.296875, 0.59375, 0.921875, 0.265625, 0.203125]
# select = [0.7]*13
# select = [0.25, 0.5, 0.5625, 0.25, 0.3125, 0.21875, 0.21875, 0.203125, 0.203125, 0.59375, 0.203125, 0.203125, 0.546875]
# get_bf_af_acc(select)

In [21]:
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

#args:gpu data_path train_batch_size eval_batch_size
class Data:
    def __init__(self, args):
        if args['gpus'] is not None:
            pin_memory = True


        transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])

        trainset = CIFAR10(root=args['data_path'], train=True, download= False, transform=transform_train)

        self.trainLoader = DataLoader(
            trainset, batch_size=args['train_batch_size'], shuffle=True,
            num_workers=args['num_workers'], pin_memory=pin_memory
        )

        testset = CIFAR10(root=args['data_path'], train=False, download= False, transform=transform_test)
        self.testLoader = DataLoader(
            testset, batch_size=args['eval_batch_size'], shuffle=False,
            num_workers=args['num_workers'], pin_memory=pin_memory)

In [22]:
# Data
print('==> Loading Data..')
args = {}
args['data_path'] = 'C:\\Users\\lenovo\\dataset\\cifar'
args['train_batch_size'] = 256
args['num_workers'] = 0
args['eval_batch_size'] = 256
args['gpus'] = 0
# args.gpus
loader = Data(args)

==> Loading Data..


In [23]:

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

    def accumulate(self, val, n=1):
        self.sum += val
        self.count += n
        if self.count > 0:
            self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    batch_size = target.size(0)
    num = output.size(1)
    target_topk = []
    appendices = []
    for k in topk:
        if k <= num:
            target_topk.append(k)
        else:
            appendices.append([0.0])
    topk = target_topk
    maxk = max(topk)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res + appendices


In [24]:

# Training
def train(model, optimizer, trainLoader, args, epoch):

    model.train()
    losses = AverageMeter()
    accurary = AverageMeter()
    print_freq = len(trainLoader.dataset) // args['train_batch_size'] // 10
    start_time = time.time()
    for batch, (inputs, targets) in enumerate(trainLoader):

        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        output = model(inputs)
        loss = loss_func(output, targets)
        loss.backward()
        losses.update(loss.item(), inputs.size(0))
        optimizer.step()

        prec1 = accuracy(output, targets)
        accurary.update(prec1[0], inputs.size(0))

        if batch % print_freq == 0 and batch != 0:
            current_time = time.time()
            cost_time = current_time - start_time
            logger.info(
                'Epoch[{}] ({}/{}):\t'
                'Loss {:.4f}\t'
                'Accurary {:.2f}%\t\t'
                'Time {:.2f}s'.format(
                    epoch, batch * args['train_batch_size'], len(trainLoader.dataset),
                    float(losses.avg), float(accurary.avg), cost_time
                )
            )
            start_time = current_time
            

#Testinga
def test(model, testLoader):
    global best_acc
    model.eval()

    losses = AverageMeter()
    accurary = AverageMeter()

    start_time = time.time()

    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testLoader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = loss_func(outputs, targets)

            losses.update(loss.item(), inputs.size(0))
            predicted = accuracy(outputs, targets)
            accurary.update(predicted[0], inputs.size(0))

        current_time = time.time()
        logger.info(
            'Test Loss {:.4f}\tAccurary {:.2f}%\t\tTime {:.2f}s\n'
            .format(float(losses.avg), float(accurary.avg), (current_time - start_time))
        )
    return accurary.avg





#         model_state_dict = model.module.state_dict() if len(args.gpus) > 1 else model.state_dict()
    
#         state = {
#             'state_dict': model_state_dict,
#             'best_acc': best_acc,
#             'optimizer': optimizer.state_dict(),
#             'scheduler': scheduler.state_dict(),
#             'epoch': epoch + 1,
#             'action': ai
#         }
#         checkpoint.save_model(state, epoch + 1, is_best
#         ckpt_dir = './checkpoints'
#         save_path = f'{ckpt_dir}/model_{epoch}.pt'
#         # print('=> Saving model to {}'.format(save_path))
#         torch.save(state, save_path)
#         if is_best:
#             shutil.copyfile(save_path, f'{self.ckpt_dir}/model_best.pt')

#     logger.info('Best accurary: {:.3f}'.format(float(best_acc)))

In [26]:
def get_logger(file_path):
    import logging 
    logger = logging.getLogger('gal')
    log_format = '%(asctime)s | %(message)s'
    formatter = logging.Formatter(log_format, datefmt='%m/%d %I:%M:%S %p')
    file_handler = logging.FileHandler(file_path)
    file_handler.setFormatter(formatter)
    stream_handler = logging.StreamHandler()
    stream_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    logger.addHandler(stream_handler)
    logger.setLevel(logging.INFO)

    return logger

## bf_acc 正比于 ft_acc吗

In [100]:
acc1_bf = [29.58,0.2946,0.2874,0.2834,0.2074,0.1856,0.1224]
action = [
    [0.75, 0.75, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.71875, 0.734375, 0.734375, 0.75, 0.5625],
    [0.75, 0.75, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.6875, 0.71875, 0.71875, 0.734375, 0.75, 0.640625],
    [0.75, 0.75, 0.6875, 0.6875, 0.6875, 0.6875, 0.65625, 0.71875, 0.71875, 0.734375, 0.734375, 0.765625, 0.53125],
    [0.625, 0.75, 0.75, 0.8125, 0.65625, 0.53125, 0.625, 0.71875, 0.78125, 0.796875, 0.609375, 0.875, 0.671875],
    [0.75, 1.0, 0.625, 0.8125, 0.8125, 0.53125, 0.78125, 0.625, 0.8125, 0.421875, 0.625, 0.734375, 0.46875],
    [0.875, 0.75, 0.9375, 0.8125, 0.9375, 0.65625, 0.28125, 0.890625, 0.21875, 0.75, 0.5, 0.203125, 0.5],
    [0.75, 0.375, 0.8125, 1.0, 0.8125, 0.53125, 0.71875, 0.875, 0.296875, 0.59375, 0.921875, 0.265625, 0.203125]
]
T = 2
ft_acc = []
logger = get_logger('./log.log')
loss_func = nn.CrossEntropyLoss()
for idx,ai in enumerate(action):
    model = get_masked_vgg(ai)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-3)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[2], gamma=0.1)
#     print(f'############# model{idx} #################')
    best_acc = 0.0
    st = time.time()
    for epoch in range(0, T):
        train(model, optimizer, loader.trainLoader, args, epoch)
        scheduler.step()
        test_acc = test(model, loader.testLoader)
        is_best = best_acc < test_acc
        best_acc = max(best_acc, test_acc)
    ed = time.time()
    logger.info(f"############# model{idx}_{T}epoch ft-acc:{best_acc} using {ed-st}s #################")
    ft_acc.append(best_acc)

01/21 06:39:03 PM | Epoch[0] (4864/50000):	Loss 0.4436	Accurary 86.43%		Time 24.78s
01/21 06:39:03 PM | Epoch[0] (4864/50000):	Loss 0.4436	Accurary 86.43%		Time 24.78s
01/21 06:39:03 PM | Epoch[0] (4864/50000):	Loss 0.4436	Accurary 86.43%		Time 24.78s
01/21 06:39:03 PM | Epoch[0] (4864/50000):	Loss 0.4436	Accurary 86.43%		Time 24.78s
01/21 06:39:03 PM | Epoch[0] (4864/50000):	Loss 0.4436	Accurary 86.43%		Time 24.78s
01/21 06:39:22 PM | Epoch[0] (9728/50000):	Loss 0.3625	Accurary 88.49%		Time 19.91s
01/21 06:39:22 PM | Epoch[0] (9728/50000):	Loss 0.3625	Accurary 88.49%		Time 19.91s
01/21 06:39:22 PM | Epoch[0] (9728/50000):	Loss 0.3625	Accurary 88.49%		Time 19.91s
01/21 06:39:22 PM | Epoch[0] (9728/50000):	Loss 0.3625	Accurary 88.49%		Time 19.91s
01/21 06:39:22 PM | Epoch[0] (9728/50000):	Loss 0.3625	Accurary 88.49%		Time 19.91s
01/21 06:39:42 PM | Epoch[0] (14592/50000):	Loss 0.3107	Accurary 90.10%		Time 20.03s
01/21 06:39:42 PM | Epoch[0] (14592/50000):	Loss 0.3107	Accurary 90.10%		Ti

In [101]:
ft_acc

[tensor(87.0900, device='cuda:0'),
 tensor(87.9600, device='cuda:0'),
 tensor(88.3100, device='cuda:0'),
 tensor(88.8200, device='cuda:0'),
 tensor(88.8400, device='cuda:0'),
 tensor(87.5800, device='cuda:0'),
 tensor(85.6700, device='cuda:0')]

In [104]:
acc1_bf

[29.58, 0.2946, 0.2874, 0.2834, 0.2074, 0.1856, 0.1224]

In [27]:
from numpy.lib.function_base import select
import torch.nn as nn
import torch
import copy
import numpy as np
from models.vgg_cifar import MaskVGG,VGG


class Pruning:
    def __init__(self,origin_model) -> None:
        super().__init__()
        self.origin_model = copy.deepcopy(origin_model)
        self.get_layer_information()

    def get_masked(self,select):
        self.all_mask = self.preprocess_get_mask(select)
        masked_vgg = MaskVGG('vgg16',select).cuda()
        self.pruned_model(masked_vgg)
        return masked_vgg

    def get_layer_information(self):
        self.prunable_idx = []
        self.prunable_ops = []
        self.layer_type_dict = {}
        self.orgin_channel = []
        self.conv_buffer_dict = {} # layer after the conv
        self.all_idx = []
        self.buffer_conv_map = {}

        i=0
        buffer_temp_idx = []
        model = self.origin_model
        modules = list(model.modules())
        n = len(modules)
        while i < n :
            m = modules[i]

            if type(m) not in [nn.Conv2d,nn.BatchNorm2d,nn.Linear,nn.AvgPool2d,nn.MaxPool2d,nn.ReLU]:
                i+=1
                continue
            else:
                assert type(m) == torch.nn.modules.conv.Conv2d
                self.prunable_ops.append(m)
                self.prunable_idx.append(i)
                self.layer_type_dict[i] = type(m)
                self.orgin_channel.append(m.out_channels) 
                self.all_idx.append(i)
                buffer_temp_idx = []
                while i != n-1:
                    i+=1
                    bu = modules[i]
                    if type(bu) is torch.nn.modules.conv.Conv2d:
                        i-=1
                        break
                    buffer_temp_idx.append(i)
                    self.all_idx.append(i)
                self.conv_buffer_dict[self.prunable_idx[-1]] = copy.deepcopy(buffer_temp_idx)
                for j in buffer_temp_idx:
                    self.buffer_conv_map[j] = self.prunable_idx[-1]
            i+=1

    # def vgg_masked(self,strategy):
    #     masked_vgg = MaskVGG('vgg16',strategy)
    #     orimo_ls = list(self.orgin_model.modules())
        
    #     for i,m in enumerate(masked_vgg.modules()):
    #         if type(m) in [nn.Conv2d,nn.BatchNorm2d,nn.Linear,nn.AvgPool2d,nn.MaxPool2d,nn.ReLU]:
    #             # type
    #             ty = type(m)

    #             # conv
    #             if ty == nn.Conv2d:
    #                 m.weight.data.copy_(orimo_ls[i].weight.data)
    #                 m.bias.data.copy_(orimo_ls[i].bias.data)
    #             # bn2d
    #             elif ty == nn.BatchNorm2d:
    #                 m.weight.data.copy_(orimo_ls[i].weight.data)
    #                 m.bias.data.copy_(orimo_ls[i].bias.data)
    #                 m.running_mean.data.copy_(orimo_ls[i].running_mean.data)
    #                 m.running_var.data.copy_(orimo_ls[i].running_var.data)
    #             elif ty == nn.Linear:
    #                 # linear
    #                 m.weight.data.copy_(orimo_ls[i].weight.data)
    #             else:# maxpool,avgpool,relu don't need params
    #                 pass
    #     masked_vgg = masked_vgg.cuda()
    #     return masked_vgg


    def pruned_model(self,masked_vgg):
        m_list = list(self.origin_model.modules())
        mp_list = list(masked_vgg.modules())
        for idx,idxx in enumerate(self.prunable_idx):
            
            # replace conv first
            mask = self.all_mask[idx]
            weight = m_list[idxx].weight.data.cpu().numpy()
            bias = m_list[idxx].bias.data.cpu().numpy()
            
            mask_weight = None
            if idx == 0:
                mask_weight = weight[mask,:,:,:]
            else:
                input_mask = self.all_mask[idx-1]
                # select input
                mask_weight = weight[:,input_mask,:,:].reshape(weight.shape[0],-1,weight.shape[2],weight.shape[3])
                # select output
                mask_weight = mask_weight[mask,:,:,:].reshape(-1,mask_weight.shape[1],mask_weight.shape[2],mask_weight.shape[3])
            mask_bias = bias[mask]
            mp = mp_list[idxx]
            mp.weight.data.copy_(torch.from_numpy(mask_weight).cuda())
            mp.bias.data.copy_(torch.from_numpy(mask_bias).cuda())

            # replace other layers
            buffer = self.conv_buffer_dict[idxx]
            for buffer_idx in buffer:
                m = m_list[buffer_idx]
                mp = mp_list[buffer_idx]
                if type(m) == nn.BatchNorm2d:
                    mp.weight.data.copy_(torch.from_numpy(m.weight.data.cpu().numpy()[mask]).cuda())
                    mp.bias.data.copy_(torch.from_numpy(m.bias.data.cpu().numpy()[mask]).cuda())
                    mp.running_mean.data.copy_(torch.from_numpy(m.running_mean.data.cpu().numpy()[mask]).cuda())
                    mp.running_var.data.copy_(torch.from_numpy(m.running_var.data.cpu().numpy()[mask]).cuda())
                elif type(m) == nn.Linear:
                    mp.weight.data.copy_(torch.from_numpy(m.weight.data.cpu().numpy()[:,mask]).cuda())
    #     print(f'replace cost {ed-st}s')
    def preprocess_get_mask(self,select,method = 'l1'):
        mask = []
        for i,a in enumerate(select):
            c = self.orgin_channel[i]
            d = int(c * a)
            mask_ = np.zeros(c,bool)
            weight = self.prunable_ops[i].weight.data.cpu().numpy()
            if method == 'l1':
                importance = np.abs(weight).sum((1, 2, 3))
                sorted_idx = np.argsort(-importance)  # sum magnitude along C_in, sort descend
                preserve_idx = sorted_idx[:d]  # to preserve index
                mask_[preserve_idx] = True
            mask.append(mask_)
        return mask

    def idx2idxx(self,idx):
        return self.prunable_idx[idx]

    def idxx2idx(self,idxx):
        return self.prunable_idx.index(idxx)


# if __name__ == "__main__":
#     origin_model = VGG('vgg16')
#     pruner = Pruning(origin_model)
#     select=[1.0]*13
#     get_acc(select)
# #     pruned_model(vgg,masked_vgg,mask)
# #     mask_model = pruner.get_masked(select=select)
# #     _validate(val_loader,mask_model)
# #     print(mask_model)


In [28]:
class Cifar10_Valider:
    def __init__(self):
        self.val_loader = self.get_split_dataset()
        
    def validate(self, model, verbose=True):
        '''
        Validate the performance on validation set
        :param val_loader:
        :param model:
        :param verbose:
        :return:
        '''
        val_loader=self.val_loader
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
        top5 = AverageMeter()

        criterion = nn.CrossEntropyLoss().cuda()
        # switch to evaluate mode
        model.eval()
        end = time.time()

        t1 = time.time()
        with torch.no_grad():
            for i, (input, target) in enumerate(val_loader):
                target = target.cuda(non_blocking=True)
                input_var = torch.autograd.Variable(input).cuda()
                target_var = torch.autograd.Variable(target).cuda()

                # compute output
                output = model(input_var)
                loss = criterion(output, target_var)

                # measure accuracy and record loss
                prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
                losses.update(loss.item(), input.size(0))
                top1.update(prec1.item(), input.size(0))
                top5.update(prec5.item(), input.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()
        t2 = time.time()
        if verbose:
            print('* Test loss: %.3f    top1: %.3f    top5: %.3f    time: %.3f' %
                  (losses.avg, top1.avg, top5.avg, t2 - t1))
    def get_split_dataset(self, batch_size=128, n_worker=0, val_size=5000, data_root='C:\\Users\\lenovo\\dataset\\cifar',\
                      shuffle=True):
        '''
            split the train set into train / val for rl search
        '''
        if shuffle:
            index_sampler = SubsetRandomSampler
        else:  # every time we use the same order for the split subset
            class SubsetSequentialSampler(SubsetRandomSampler):
                def __iter__(self):
                    return (self.indices[i] for i in torch.arange(len(self.indices)).int())
            index_sampler = SubsetSequentialSampler

#         print('=> Preparing data: {}...'.format(dset_name))
        
#         transform_train = transforms.Compose([
#             transforms.RandomCrop(32, padding=4),
#             transforms.RandomHorizontalFlip(),
#             transforms.ToTensor(),
#             transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#         ])
        transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
        ])
#         trainset = torchvision.datasets.CIFAR10(root=data_root, train=True, download=True, transform=transform_train)

        valset = torchvision.datasets.CIFAR10(root=data_root, train=False, download=True, transform=transform_test)
        n_val = len(valset)
        assert val_size < n_val
        indices = list(range(n_val))
        np.random.shuffle(indices)
        _, val_idx = indices[val_size:], indices[:val_size]
#         train_idx = list(range(len(trainset)))  # all train set for train

#         train_sampler = index_sampler(train_idx)
        val_sampler = index_sampler(val_idx)

#         train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, sampler=train_sampler,
#                                                    num_workers=n_worker, pin_memory=True)
        val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, sampler=val_sampler,
                                                 num_workers=n_worker, pin_memory=True)
        n_class = 10

        return val_loader

In [29]:
cv = Cifar10_Valider()
# cv.validate(get_masked_vgg([1.0]*13))

Files already downloaded and verified


In [30]:
# mask = preprocess_get_mask(select)
# from models.vgg_cifar import MaskVGG
# masked_vgg = MaskVGG('vgg16',select).cuda()
# _validate(val_loader,masked_vgg)
# pruned_model(vgg,masked_vgg,mask)
# _validate(val_loader,masked_vgg)

In [31]:
def get_reduced_rate(action):
    macs, params = get_fp_from_action(action)
    macs_orginal, params_orginal = get_fp_from_action([1.0]*13)
    print('preserve:')
    print(macs/macs_orginal,params/params_orginal)
    print('pruned:')
    print(1-macs/macs_orginal,1-params/params_orginal)
#     return macs/macs_orginal,params/params_orginal
    
    

In [32]:
def get_flops_params_from_model(model):
    from thop import profile
    import torch
    input = torch.randn(1, 3, 32, 32)
    macs, params = profile(model, inputs=(input, ))
#     print(macs/1e6, params/1e6)
    return macs/1e6, params/1e6

In [33]:
def get_fp_from_action(action):
    mask = preprocess_get_mask(action)
    from models.vgg_cifar import MaskVGG
    masked_vgg = MaskVGG('vgg16',action)
#     print()
    return get_flops_params_from_model(masked_vgg)

In [34]:
# action = [0.75, 0.625, 1.0, 0.25, 0.90625, 0.34375, 0.90625, 0.421875, 0.625, 0.203125, 0.765625, 0.203125, 0.40625]
# get_fp_from_action(action)

In [35]:
# action = [0.75, 0.625, 1.0, 0.25, 0.90625, 0.34375, 0.90625, 0.421875, 0.625, 0.203125, 0.765625, 0.203125, 0.40625]
# get_reduced_rate(action)

In [36]:
# get_acc([0.25,0.5,0.5625,0.1875,0.9375,0.25,0.15625,0.109375,0.109375,0.28125,0.109375,0.109375,0.703125])

In [168]:
# dic = torch.load('./logs/vgg16_cifar10_finetune-run56/ckpt.best.pth.tar')
# vgg16_30 = MaskVGG('vgg16',[0.75, 0.625, 1.0, 0.25, 0.90625, 0.34375, 0.90625, 0.421875, 0.625, 0.203125, 0.765625, 0.203125, 0.40625])
# vgg16_30.load_state_dict(dic['state_dict'])

In [17]:
import numpy as np
np.random.choice(range(3),1)[0]

2

In [8]:
import torch
m = torch.nn.Conv2d(in_channels=3,out_channels=1,kernel_size=3,padding=1,bias=False)

In [9]:
m.weight

Parameter containing:
tensor([[[[-0.1574,  0.1919, -0.0597],
          [ 0.0149,  0.1871, -0.0072],
          [ 0.0056,  0.0513, -0.0983]],

         [[-0.0168, -0.1040, -0.1004],
          [-0.0662,  0.1262, -0.0367],
          [-0.0690,  0.1048, -0.0518]],

         [[-0.0606,  0.0876,  0.0088],
          [-0.0724,  0.0696,  0.1688],
          [ 0.1339, -0.0232,  0.0223]]]], requires_grad=True)

In [104]:
input = torch.randn((50,3,2,2))
# input
input_pad = torch.nn.functional.pad(input,pad=(1,1,1,1),value=0)

In [105]:
m = torch.nn.Conv2d(in_channels=3,out_channels=1,kernel_size=3,padding=1,bias=True)

In [106]:
y = m(input)

In [112]:
torch.sum(input_pad[:,:,0:3,0:3]  * m.weight,dim=(3,2,1))+ m.bias.data[0]

tensor([ 0.2503,  0.1073,  0.1506, -0.4082,  0.1296, -0.4880, -0.0871, -0.6540,
        -0.3553,  0.0674,  0.3267, -0.2391, -0.0998,  0.9294,  0.0316, -0.1539,
         0.2457,  0.8251, -0.2070,  0.6967, -0.7093,  1.0018, -0.0552, -0.1139,
         0.1567,  0.1967, -1.2273, -0.2346,  0.5532, -0.3946, -0.1133, -0.9122,
        -0.3404, -0.3401, -0.6173,  0.3668, -0.6848, -0.2288, -0.0215,  0.1898,
        -0.2610,  0.4238,  0.2819, -0.7106, -0.3907, -0.7498,  0.1644,  0.4771,
        -0.0038,  0.3640], grad_fn=<AddBackward0>)

In [107]:

torch.sum(input_pad[:,:,0:3,0:3]  * m.weight)+ m.bias.data[0]


tensor(-2.0289, grad_fn=<AddBackward0>)

In [100]:
y[:,:,0,0]

tensor([[-0.3911]], grad_fn=<SelectBackward>)

In [48]:
input_pad.shape

torch.Size([1, 3, 4, 4])

In [49]:
y = m(input)
y

tensor([[[[0.5848, 0.4401],
          [0.1162, 0.4401]]]], grad_fn=<ThnnConv2DBackward>)

In [50]:
y[:,:,0,1]

tensor([[0.4401]], grad_fn=<SelectBackward>)

In [33]:
input_pad[:,:,:3,:3]

torch.Size([1, 3, 3, 3])

In [35]:
m.weight.shape

torch.Size([1, 3, 3, 3])

tensor(-0.0238, grad_fn=<SumBackward0>)

In [116]:
sd = torch.load(r'C:\Users\lenovo\Desktop\cacp\cacp_vgg\checkpoints\vgg16_cifar10.pt')

In [118]:
# sd

In [147]:
a = [3,64, 64, 'M', 64, 128, 'M', 128, 256, 256, 'M', 256, 512, 512, 'M', 512, 512, 512, 'M']

In [154]:
len([1.0,0.3,0.3,0.3,0.3,0.3,0.3,0.1,0.2,0.3,0.3,0.3,0.3])

13

In [157]:
m.weight.sum((0, 2, 3))

tensor([-0.3530,  0.2482,  0.4256], grad_fn=<SumBackward1>)

In [177]:
B = nn.Linear(512,10)

In [181]:
B.weight.shape

torch.Size([10, 512])

In [23]:
import torch
m = torch.nn.Conv2d(in_channels=3,out_channels=4,kernel_size=3,padding=1,bias=True)

In [24]:

mask = np.zeros(4,bool)

In [25]:
m.bias.shape

torch.Size([4])

In [34]:
X = torch.randn((200,27))
Y = torch.randn((200,10))
from sklearn.linear_model import LinearRegression
reg = LinearRegression(fit_intercept=False)
reg.fit(X, Y)


LinearRegression(fit_intercept=False)

In [35]:
W = reg.coef_

In [36]:
W.reshape((10,3,3,3))

array([[[[ 3.44062261e-02, -2.53497902e-03, -4.46233824e-02],
         [ 2.72693541e-02, -2.63566840e-02, -1.00391440e-01],
         [-1.66097984e-01,  7.32494742e-02, -4.23325300e-02]],

        [[-2.20595021e-02, -6.38243034e-02,  1.85469389e-02],
         [-5.87097090e-03,  6.67332709e-02,  7.68231750e-02],
         [ 9.88645628e-02,  1.20755650e-01, -2.03198493e-02]],

        [[ 9.13487934e-03, -4.31717336e-02,  4.07561511e-02],
         [ 6.15837350e-02,  6.99409619e-02, -5.98619059e-02],
         [ 1.43059805e-01, -1.03198156e-01, -9.30793397e-03]]],


       [[[ 1.27959207e-01,  8.28858465e-03, -2.31272236e-01],
         [ 7.38568678e-02, -2.65119337e-02, -5.69740459e-02],
         [ 3.60761657e-02,  2.94754300e-02,  9.32743549e-02]],

        [[ 1.87041193e-01, -1.05630651e-01, -5.53176692e-03],
         [-1.98659282e-02, -3.49560343e-02,  5.87630868e-02],
         [ 9.93013233e-02,  6.64867088e-02,  7.84636587e-02]],

        [[-2.58530471e-02,  6.37312829e-02, -7.40159601e-0